In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("../data/train.csv",index_col = "item_id", parse_dates = ["activation_date"])
test_df = pd.read_csv("../data/test.csv",index_col = "item_id", parse_dates = ["activation_date"])

In [3]:
#Prepare data
N_A_word = "Неопределено"
test_df["param_1"].fillna(N_A_word, inplace=True)
test_df["param_2"].fillna(N_A_word, inplace=True)
test_df["param_3"].fillna(N_A_word, inplace=True)
#TBD заполнить иерархически
test_df["description"].fillna(N_A_word, inplace=True)
train_df["param_1"].fillna(N_A_word, inplace=True)
train_df["param_2"].fillna(N_A_word, inplace=True)
train_df["param_3"].fillna(N_A_word, inplace=True)
train_df["description"].fillna(N_A_word, inplace=True)

In [8]:
train_df["full_prices"] = train_df.groupby(["category_name", "param_1"])["price"].apply(lambda x: x.fillna(x.mean()))
test_df["full_prices"] = test_df.groupby(["category_name", "param_1"])["price"].apply(lambda x: x.fillna(x.mean()))
test_df.drop(["price"], axis=1, inplace=True)
train_df.drop(["price"], axis=1, inplace=True);
train_df["full_prices"].fillna(0, inplace=True)
test_df["full_prices"].fillna(0, inplace=True)

In [10]:
train_df[train_df["full_prices"]==0]
# TODO fill zero prices with bad cost values

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,full_prices
item_id,,,,,,,,,,,,,,,,,
97e5dba1a544,ba77f4b48755,Новосибирская область,Новосибирск,Услуги,Предложение услуг,Другое,Неопределено,Неопределено,Обмен места в детском саду,Обменяю место в детском саду N72 на детский са...,4,2017-03-28,Private,NaN,NaN,0.60000,0.0
35f011d65bb8,dfa988975741,Ростовская область,Новочеркасск,Животные,Кошки,Другая,Неопределено,Неопределено,Кошечка ищет дом,Рыжее счастье дом и любящих хозяев .Кошечке 1 ...,15,2017-03-28,Private,88c70b8941bd4230bd9938a038aeb7998d097de7d4d717...,2063.0,0.78503,0.0
54d642ff229a,2f9b389a4cf0,Воронежская область,Воронеж,Услуги,Предложение услуг,Другое,Неопределено,Неопределено,Подключение uber (Воронеж),Предлагаем работу водителем (сменный график)./...,10,2017-03-21,Company,d3f60b28ba7072f9722c98d668918df864f5ee160eb786...,2266.0,0.00000,0.0
db6143e5d1b4,8467734ce392,Краснодарский край,Сочи,Хобби и отдых,Книги и журналы,Учебная литература,Неопределено,Неопределено,Учебники 8 класса,Неопределено,58,2017-03-20,Private,ecf8e9498c71bb84206936ef1cceade666818e024a9e99...,1050.0,0.18907,0.0
ee1d51eb4e6c,c3e00b4a0f99,Иркутская область,Иркутск,Животные,Кошки,Другая,Неопределено,Неопределено,Отдам милых котят,"Отдам очень милых котят. Им 1,5 месяца приучен...",35,2017-03-25,Private,2cba8460919c67557be36e9add6fee3002f1afe61e08fd...,2054.0,0.39355,0.0
c5bca1a63c11,c5d59c758a04,Башкортостан,Кумертау,Животные,Кошки,Британская,Неопределено,Неопределено,Отдадим кота в хорошие руки,Отдадим котика в хорошие руки в связи с семейн...,5,2017-03-16,Private,e138e00cea4979c7894036c16f6708bcda1d54a5b580c0...,2054.0,0.78503,0.0
1c10b9025e48,add013223f9b,Самарская область,Самара,Услуги,Предложение услуг,Уборка,Вывоз мусора,Неопределено,"Вывезу мусор камаз,ЗИЛ,газель","УСЛУГИ по ВЫВОЗу МУСОРа, УСЛУГИ ГРУЗЧИКОВ, ПРИ...",2,2017-03-18,Private,4d5bcf916ebdd50eed2c51e5414674c8578df7420a02b1...,1207.0,0.40000,0.0
a3d1d1ba909e,ecf3f232be6f,Татарстан,Казань,Для дома и дачи,Растения,Неопределено,Неопределено,Неопределено,Бесплатно отдаю растения,"На площадке по адресу г. Казань, Ул. Четаева, ...",61,2017-03-15,Private,a4d46e4166a6699dd5af1bb3140b1b07cbde237c6a08ec...,1733.0,0.38925,0.0
b65c8ea48f4e,3025f6c20e3d,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Подставки и тумбы,Неопределено,Неопределено,Тумба бесплатно,Отдам бесплатно.В реальности смотрится лучше.,236,2017-03-22,Private,19fcbf94ae776a5fe49b74aeb18cff9a221af56d3fad7b...,1456.0,0.86521,0.0


In [24]:
def create_time_vars(df):
    df["Weekday"] = df['activation_date'].dt.weekday
    df["Weekd of Year"] = df['activation_date'].dt.week
    df["Day of Month"] = df['activation_date'].dt.day
create_time_vars(train_df)
create_time_vars(test_df)
train_df.drop(["activation_date"],axis=1,inplace=True)
test_df.drop(["activation_date"],axis=1,inplace=True)

# Prepare data

In [70]:
train_df['norm_price'] = pd.Series(0,index=train_df.index)
test_df['norm_price'] = pd.Series(0,index=test_df.index)

In [42]:
y = train_df["deal_probability"]
train_df.drop(["deal_probability"], axis=1, inplace=True)

In [11]:
from sklearn.preprocessing import StandardScaler
categories = train_df["category_name"].unique()
for cat in categories:
    scaler = StandardScaler()
    norm_price = scaler.fit_transform(train_df[train_df["category_name"]==cat]["full_prices"].values.reshape(-1,1))
    train_df.loc[train_df["category_name"]==cat,"norm_price"] = norm_price
    norm_price_test = scaler.transform(test_df[test_df["category_name"]==cat]["full_prices"].values.reshape(-1,1))
    test_df.loc[test_df["category_name"]==cat,"norm_price"] = norm_price_test

In [13]:
train_df.drop(["full_prices"], axis = 1, inplace=True)
test_df.drop(["full_prices"], axis = 1, inplace=True)

In [16]:
train_has_image = train_df["image"].notnull().astype('int')
test_has_image = test_df["image"].notnull().astype('int')

In [17]:
from sklearn.preprocessing import LabelBinarizer
usertLab = LabelBinarizer()
train_usert_oh = usertLab.fit_transform(train_df["user_type"])
test_usert_oh = usertLab.transform(test_df["user_type"])

In [18]:
def get_train_test_oh(column, sparse=True):
    labeler = LabelBinarizer(sparse_output=sparse)
    train_lbl_oh = labeler.fit_transform(train_df[column])
    test_lbl_oh = labeler.transform(test_df[column])
    return train_lbl_oh, test_lbl_oh

train_pr1_oh, test_pr1_oh = get_train_test_oh("param_1")

In [19]:
train_cat_oh, test_cat_oh = get_train_test_oh("category_name")

In [20]:
train_pcat_oh, test_pcat_oh = get_train_test_oh("parent_category_name")

In [21]:
train_city_oh, test_city_oh = get_train_test_oh("city")

In [22]:
train_region_oh, test_region_oh = get_train_test_oh("region")

In [25]:
train_wd_oh, test_wd_oh = get_train_test_oh("Weekday")

In [26]:
train_week_oh, test_week_oh = get_train_test_oh("Weekd of Year")

In [27]:
train_day_oh, test_day_oh = get_train_test_oh("Day of Month")

In [28]:
def free_in(v):
    if "бесплатн" in v:
        return 1
    else: 
        return 0
train_free_title = train_df["title"].apply(free_in).values.reshape(-1,1)
train_free_desc = train_df["description"].apply(free_in).values.reshape(-1,1)
test_free_title = test_df["title"].apply(free_in).values.reshape(-1,1)
test_free_desc = test_df["description"].apply(free_in).values.reshape(-1,1)

In [29]:
train_df["image_top_1"].fillna(0, inplace=True)
test_df["image_top_1"].fillna(0, inplace=True)

In [30]:
train_img_cls, test_img_cls = get_train_test_oh("image_top_1")

In [31]:
train_pr2_oh, test_pr2_oh = get_train_test_oh("param_2")

In [32]:
train_pr3_oh, test_pr3_oh = get_train_test_oh("param_3")

In [33]:
train_pr3_oh

<1503424x1220 sparse matrix of type '<class 'numpy.int64'>'
	with 1503424 stored elements in Compressed Sparse Row format>

# Text Features

In [35]:
train_titles = train_df["title"]
test_titles = test_df["title"]
train_desc = train_df["description"]
test_desc = test_df["description"]

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_vct_title = TfidfVectorizer()
train_title_tfidf = train_vct_title.fit_transform(train_titles)
test_title_tfidf = train_vct_title.transform(test_titles)

In [37]:
train_vct_desc = TfidfVectorizer()
train_desc_tfidf = train_vct_desc.fit_transform(train_desc)
test_desc_tfidf = train_vct_desc.transform(test_desc)

In [38]:
from scipy.sparse import hstack
train_text_x = hstack([train_title_tfidf, train_desc_tfidf])
test_text_x = hstack([test_title_tfidf, test_desc_tfidf])

In [43]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
rcl_text = Ridge(random_state=17)
scores = cross_val_score(rcl_text, train_text_x, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.8min finished


In [44]:
np.mean(np.sqrt(-scores))

0.23578176616430954

# Combine data

In [48]:
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,item_seq_number,user_type,image,image_top_1,norm_price,Weekday,Weekd of Year,Day of Month
item_id,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,Неопределено,Неопределено,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",2,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,-0.027701,1,13,28


In [49]:
from scipy.sparse import hstack
train_x = hstack([train_pr3_oh,train_pr2_oh,train_img_cls,train_free_title, train_free_desc,train_df["norm_price"].values.reshape(-1,1),train_cat_oh, train_city_oh, train_has_image.values.reshape(-1,1), train_pcat_oh, train_pr1_oh, train_region_oh, train_usert_oh])

In [50]:
test_x = hstack([test_pr3_oh,test_pr2_oh,test_img_cls,test_free_title, test_free_desc,test_df["norm_price"].values.reshape(-1,1),test_cat_oh, test_city_oh, test_has_image.values.reshape(-1,1), test_pcat_oh, test_pr1_oh, test_region_oh, test_usert_oh])

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.2, random_state=42)

# Catboost training

In [53]:
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,item_seq_number,user_type,image,image_top_1,norm_price,Weekday,Weekd of Year,Day of Month
item_id,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,Неопределено,Неопределено,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",2,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,-0.027701,1,13,28


In [55]:
cat_features_train = train_df.drop(["user_id","title","description","item_seq_number","image"], axis=1)
cat_features_test = test_df.drop(["user_id","title","description","item_seq_number","image"], axis=1)

In [57]:
cat_features_train.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'user_type', 'image_top_1', 'Weekday',
       'Weekd of Year', 'Day of Month'],
      dtype='object')

In [67]:
from catboost import Pool, CatBoostRegressor

train_pool = Pool(cat_features_train, y, cat_features=list(range(len(cat_features_train.columns)-1)))
test_pool = Pool(cat_features_test, cat_features=list(range(len(cat_features_test.columns)-1))) 

# specify the training parameters 
model = CatBoostRegressor(iterations=50, depth=3, learning_rate=0.5, loss_function='RMSE')

In [81]:
from catboost import cv
parameters = {'iterations': 50, 'depth':3, 'learning_rate':0.5, 'loss_function':'RMSE'}
cb_scores = cv(params=parameters, pool=train_pool)
np.sqrt(cb_scores)

0:	learn: 0.2575385	test: 0.2575275	best: 0.2575275 (0)	total: 3.07s	remaining: 2m 30s
1:	learn: 0.2447862	test: 0.2447387	best: 0.2447387 (1)	total: 5.64s	remaining: 2m 15s
2:	learn: 0.2406174	test: 0.2405224	best: 0.2405224 (2)	total: 8.15s	remaining: 2m 7s
3:	learn: 0.2388252	test: 0.2386645	best: 0.2386645 (3)	total: 10.5s	remaining: 2m
4:	learn: 0.2379968	test: 0.2378290	best: 0.2378290 (4)	total: 13.5s	remaining: 2m 1s
5:	learn: 0.2372764	test: 0.2370775	best: 0.2370775 (5)	total: 16.1s	remaining: 1m 57s
6:	learn: 0.2367532	test: 0.2365049	best: 0.2365049 (6)	total: 18.1s	remaining: 1m 51s
7:	learn: 0.2364485	test: 0.2361992	best: 0.2361992 (7)	total: 20.5s	remaining: 1m 47s
8:	learn: 0.2360688	test: 0.2357952	best: 0.2357952 (8)	total: 22.9s	remaining: 1m 44s
9:	learn: 0.2358607	test: 0.2355649	best: 0.2355649 (9)	total: 26.2s	remaining: 1m 44s
10:	learn: 0.2356280	test: 0.2353121	best: 0.2353121 (10)	total: 28.7s	remaining: 1m 41s
11:	learn: 0.2354121	test: 0.2350760	best: 0.23

,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0.507472,0.024711,0.507483,0.018154
1,0.494711,0.023837,0.494759,0.018561
2,0.490431,0.026886,0.490528,0.018597
3,0.488533,0.025358,0.488697,0.017223
4,0.487677,0.024968,0.487849,0.017973
5,0.486906,0.028952,0.487110,0.017137
6,0.486318,0.024757,0.486573,0.019766
7,0.486003,0.025967,0.486260,0.018666
8,0.485587,0.023051,0.485869,0.019427
9,0.485350,0.024076,0.485655,0.018749


In [68]:
model.fit(train_pool)
cb_preds = model.predict(test_pool)

0:	learn: 0.2574906	total: 778ms	remaining: 38.1s
1:	learn: 0.2447709	total: 1.29s	remaining: 30.9s
2:	learn: 0.2408886	total: 1.97s	remaining: 30.9s
3:	learn: 0.2389303	total: 2.62s	remaining: 30.2s
4:	learn: 0.2381802	total: 3.11s	remaining: 28s
5:	learn: 0.2369230	total: 3.74s	remaining: 27.4s
6:	learn: 0.2365194	total: 4.36s	remaining: 26.8s
7:	learn: 0.2362227	total: 4.89s	remaining: 25.7s
8:	learn: 0.2358373	total: 5.67s	remaining: 25.8s
9:	learn: 0.2357008	total: 6.38s	remaining: 25.5s
10:	learn: 0.2354876	total: 7.24s	remaining: 25.7s
11:	learn: 0.2352836	total: 7.82s	remaining: 24.8s
12:	learn: 0.2350241	total: 8.56s	remaining: 24.4s
13:	learn: 0.2348193	total: 9.22s	remaining: 23.7s
14:	learn: 0.2347478	total: 9.94s	remaining: 23.2s
15:	learn: 0.2345717	total: 10.6s	remaining: 22.5s
16:	learn: 0.2343808	total: 11.2s	remaining: 21.8s
17:	learn: 0.2342854	total: 11.7s	remaining: 20.9s
18:	learn: 0.2342085	total: 12.6s	remaining: 20.5s
19:	learn: 0.2341028	total: 13.4s	remaining

# Training

In [72]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
import numpy as np
rcl = Ridge(random_state=101)
scores = cross_val_score(rcl, X_train, y_train, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")
np.mean(np.sqrt(-scores))

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  4.9min finished


0.2332616851382691

In [73]:
import numpy as np
np.mean(np.sqrt(-scores))

0.2332616851382691

In [74]:
from sklearn.model_selection import cross_val_predict
ridge_scores = cross_val_predict(rcl, train_x, y, n_jobs = -1, verbose=1)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  6.9min finished


In [75]:
text_scores = cross_val_predict(rcl, train_text_x, y, n_jobs = -1, verbose=1)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.0min finished


In [82]:
cb_scores

,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0.257527,0.000611,0.257539,0.000330
1,0.244739,0.000568,0.244786,0.000345
2,0.240522,0.000723,0.240617,0.000346
3,0.238664,0.000643,0.238825,0.000297
4,0.237829,0.000623,0.237997,0.000323
5,0.237077,0.000838,0.237276,0.000294
6,0.236505,0.000613,0.236753,0.000391
7,0.236199,0.000674,0.236449,0.000348
8,0.235795,0.000531,0.236069,0.000377
9,0.235565,0.000580,0.235861,0.000352


In [78]:
combined_scores = np.concatenate([ridge_scores.reshape(-1,1),text_scores.reshape(-1,1),cb_preds.reshape(-1,1)], axis=1)
combined_scores.shape

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [46]:
from xgboost import XGBRegressor
xgrgr = XGBRegressor()
xgrgr.fit(combined_scores, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [48]:
"""
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
alphas = [0.1, 0.5,1.0, 2.0,5.0]
for alpha in alphas:
    rcl = Ridge(random_state=101, alpha=alpha)
    scores = cross_val_score(rcl, train_x, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")
    print("%s:%s"% (alpha, np.mean(np.sqrt(-scores))))"""

'\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.linear_model import Ridge\nalphas = [0.1, 0.5,1.0, 2.0,5.0]\nfor alpha in alphas:\n    rcl = Ridge(random_state=101, alpha=alpha)\n    scores = cross_val_score(rcl, train_x, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")\n    print("%s:%s"% (alpha, np.mean(np.sqrt(-scores))))'

In [49]:
import numpy as np

In [50]:
final_ridge = Ridge(random_state=101)
final_ridge.fit(train_x, y)
y_pred = final_ridge.predict(test_x)

In [51]:
final_ridge_text = Ridge(random_state=17)
final_ridge_text.fit(train_text_x, y)
y_pred_text = final_ridge_text.predict(test_text_x)

In [52]:
test_item_id = main_test_df.index

In [53]:
y_test_final = np.concatenate([y_pred.reshape(-1,1),y_pred_text.reshape(-1,1)], axis=1)

In [54]:
lgb_score = xgrgr.predict(y_test_final)

In [55]:
sub_df = pd.DataFrame({"item_id":test_item_id})
sub_df["deal_probability"] = lgb_score.clip(0,1)
sub_df.to_csv("/tmp/baseline_avito_xgb.csv", index=False)

In [56]:
lgb_score.shape


(508438,)

In [57]:
y_pred.shape

(508438,)